# Initialization

Test notebook for the C-MAPPS benchmark. Get best parameters for each dataset. 

First we import the necessary packages and create the global variables.

In [1]:
import math
import time
import numpy as np
import csv
from data_handler_CMAPS import CMAPSDataHandler
from tunable_model import SequenceTunableModelRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler

from tunable_model import SequenceTunableModelRegression

from keras.models import Sequential, Model
from keras.layers import Dense, Input, Dropout, Reshape, Conv2D, Flatten, MaxPooling2D
from keras.optimizers import Adam
from keras.callbacks import LearningRateScheduler
from keras import backend as K
from keras import regularizers

from scipy.optimize import differential_evolution

import CMAPSAuxFunctions

Using TensorFlow backend.


# Define the model

In [2]:
def RULmodel_SN(input_shape):
    
    lambda_regularization = 0.20
    
    #Create a sequential model
    model = Sequential()
    
    #Add the layers for the model
    model.add(Dense(20, input_dim=input_shape, activation='relu', kernel_initializer='glorot_normal', 
                    kernel_regularizer=regularizers.l2(lambda_regularization), name='fc1'))
    model.add(Dense(1, activation='linear', name='out'))
    
    return model

# Create the Data Handler

In [3]:
#Selected as per CNN paper
features = ['T2', 'T24', 'T30', 'T50', 'P2', 'P15', 'P30', 'Nf', 'Nc', 'epr', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'farB', 'htBleed', 'Nf_dmd', 'PCNfR_dmd', 'W31', 'W32']
selected_indices = np.array([2, 3, 4, 7, 8, 9, 11, 12, 13, 14, 15, 17, 20, 21])
selected_features = list(features[i] for i in selected_indices-1)
data_folder = '../CMAPSSData'

window_size = 30
window_stride = 1
max_rul = 125

min_max_scaler = MinMaxScaler(feature_range=(-1, 1))

dHandler_cmaps = CMAPSDataHandler(data_folder, 1, selected_features, max_rul, window_size, 
                                  window_stride, data_scaler=min_max_scaler)
#dHandler_cmaps.load_data(verbose=1, cross_validation_ratio=0.2)

# Create theTunable Model

In [4]:
min_max_scaler = MinMaxScaler(feature_range=(-1, 1))
#min_max_scaler = None

In [5]:
def get_compiled_model(shape, model_type='ann'):

    #Shared parameters for the models
    optimizer = Adam(lr=0, beta_1=0.5)
    lossFunction = "mean_squared_error"
    metrics = ["mse"]
    model = None

    #Create and compile the models

    if model_type=='ann':
        model = RULmodel_SN(shape)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)
    elif model_type=='lstm':
        """nFeatures = len(selected_features)
        shapeLSTM = (window_size, nFeatures)
        model = RULmodel_LSTM(shapeLSTM)
        model.compile(optimizer = optimizer, loss = lossFunction, metrics = metrics)"""
        pass
    else:
        pass

    return model

In [6]:
nFeatures = len(selected_features)
shapeSN = nFeatures*window_size
modelRULSN = get_compiled_model(shapeSN, model_type='ann')
tModel = SequenceTunableModelRegression('ModelRUL_SN_1', modelRULSN, lib_type='keras', data_handler=dHandler_cmaps)
#tModel = SequenceTunableModelRegression('ModelRUL_LSTM_1', modelRULLSTM, lib_type='keras', data_handler=dHandler_cmaps)
tModel.data_handler.data_scaler = min_max_scaler

# Define objective function

Define the function that evaluates each set of data-related params and returns the RMSE as value.

In [7]:
def nn_optmize_fun(x, tModel, verbose=0, epochs=250, saveToFile = None, iterations = 0):
    
    #Clear the previous tensorflow graph
    K.clear_session()
    
    maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
    
    #Extract the tunning variables from the input vector
    #Round the values to the nearest integer since this implementation is for real numbers
    x = x.astype(int)

    #load the data using the selected parameters
    tModel.data_handler.sequence_length = x[0]
    #tModel.data_handler.sequence_length = maxWindowSize[datasetNumber]
    tModel.data_handler.sequence_stride = x[1]
    tModel.data_handler.max_rul = x[2]
    
    tModel.load_data(unroll=True, verbose=1, cross_validation_ratio=0)
    
    if iterations == 0:
        print("Training model")
    
    #Create new model
    lrate = LearningRateScheduler(CMAPSAuxFunctions.step_decay)
    nFeatures = len(selected_features)
    model_shape = nFeatures*tModel.data_handler.sequence_length
    modelRULSN = get_compiled_model(model_shape, model_type='ann')
    tModel.change_model('ModelRUL_SN_1', modelRULSN, 'keras')
    
    #Train model
    tModel.train_model(learningRate_scheduler=lrate, verbose=1)
    time = tModel.train_time
    
    if iterations == 0:
        print("Training time {}".format(time))
    
    if iterations == 0:
        print("Assesing model performance")
    #Assess the model performance
    tModel.evaluate_model(["rhs"], round=2)
    cScores = tModel.scores
    rmse = math.sqrt(cScores['score_1'])
    rhs = cScores['rhs']
    #print("The score for this model is: {}".format(rmse))
    
    msgStr = "The model variables are " + str(x) + "\tThe scores are: [RMSE:{:.4f}, RHS:{:.4f}]\n".format(rmse, rhs)
    row = x.tolist() + [rmse, rhs]
    
    if saveToFile is not None:
        #print(msgStr)
        writer = csv.writer(saveToFile)
        #row = x.append(rmse)
        #row = x.append(rhs)
        writer.writerow(row)
        #saveToFile.write(msgStr)
    else:
        print(row)
    
    #Return RMSE as the performance metric to steer the search
    return rmse
    

# Optimize the parameters for the NN using DE

In [8]:
#Optimize the parameters for the NN using DE

#maxWindowSize = {'1':30, '2':20, '3':30, '4':18}
maxWindowSize = {'1':30, '2':20} #Do it only for datasets 1 and 2
totalTime = {'1':0, '2':0, '3':0, '4':0}
results = {'1':0, '2':0, '3':0, '4':0}

#datasetNumber = '1'

selected_features = ['T24', 'T30', 'T50', 'P30', 'Nf', 'Nc', 'Ps30', 'phi', 'NRf', 'NRc', 
                     'BPR', 'htBleed', 'W31', 'W32']



for datasetNumber in maxWindowSize:
    
    print("Tunning for dataset "+datasetNumber)
    file = open("results/MLP/intermediateResults_refactorized_"+datasetNumber+".csv", "w")

    windowSizeBounds = [1,maxWindowSize[datasetNumber]]
    windowStrideBounds = [1,10]
    constantRULBounds = [90,140]

    bounds = [windowSizeBounds, windowStrideBounds, constantRULBounds]
    #bounds = [windowStrideBounds, constantRULBounds] #Optimize only 2 variabes
    
    tModel.data_handler.change_dataset(datasetNumber)

    startTime = time.clock()
    tempResults = differential_evolution(nn_optmize_fun, bounds, 
                                     args=(tModel, 0, 20, file, 1),
                                    strategy='best1bin', maxiter=30, popsize=4, disp=True, polish=False)
    results[datasetNumber] = tempResults
    endTime = time.clock()

    file.close()
    totalTime[datasetNumber] = endTime - startTime

Tunning for dataset 2
Loading data for dataset 2 with window_size of 9, stride of 5 and maxRUL of 91. Cros-Validation ratio 0
Loading data from file and computing dataframes
Epoch 1/250
10447/10447 [==============================] - 0s 47us/step - loss: 5535.4322 - mean_squared_error: 5528.4105
Epoch 2/250
10447/10447 [==============================] - 0s 2us/step - loss: 5164.0660 - mean_squared_error: 5156.8241
Epoch 3/250
10447/10447 [==============================] - 0s 3us/step - loss: 4721.1045 - mean_squared_error: 4713.3754
Epoch 4/250
10447/10447 [==============================] - 0s 2us/step - loss: 4243.4526 - mean_squared_error: 4234.9583
Epoch 5/250
10447/10447 [==============================] - 0s 2us/step - loss: 3762.7519 - mean_squared_error: 3753.2385
Epoch 6/250
10447/10447 [==============================] - 0s 2us/step - loss: 3303.7071 - mean_squared_error: 3292.9685
Epoch 7/250
10447/10447 [==============================] - 0s 2us/step - loss: 2884.4745 - mean_squ

10447/10447 [==============================] - 0s 2us/step - loss: 742.6820 - mean_squared_error: 702.9828
Epoch 68/250
10447/10447 [==============================] - 0s 2us/step - loss: 732.2042 - mean_squared_error: 691.5648
Epoch 69/250
10447/10447 [==============================] - 0s 2us/step - loss: 721.7205 - mean_squared_error: 680.0917
Epoch 70/250
10447/10447 [==============================] - 0s 2us/step - loss: 711.7587 - mean_squared_error: 669.2131
Epoch 71/250
10447/10447 [==============================] - 0s 2us/step - loss: 701.7700 - mean_squared_error: 658.2822
Epoch 72/250
10447/10447 [==============================] - 0s 2us/step - loss: 692.0735 - mean_squared_error: 647.6465
Epoch 73/250
10447/10447 [==============================] - 0s 2us/step - loss: 682.7607 - mean_squared_error: 637.3936
Epoch 74/250
10447/10447 [==============================] - 0s 2us/step - loss: 673.3613 - mean_squared_error: 627.0389
Epoch 75/250
10447/10447 [===========================

10447/10447 [==============================] - 0s 2us/step - loss: 475.6848 - mean_squared_error: 407.1526
Epoch 136/250
10447/10447 [==============================] - 0s 2us/step - loss: 475.2014 - mean_squared_error: 406.6204
Epoch 137/250
10447/10447 [==============================] - 0s 2us/step - loss: 474.7302 - mean_squared_error: 406.0954
Epoch 138/250
10447/10447 [==============================] - 0s 2us/step - loss: 474.2770 - mean_squared_error: 405.5915
Epoch 139/250
10447/10447 [==============================] - 0s 2us/step - loss: 473.7950 - mean_squared_error: 405.0516
Epoch 140/250
10447/10447 [==============================] - 0s 2us/step - loss: 473.3166 - mean_squared_error: 404.5256
Epoch 141/250
10447/10447 [==============================] - 0s 2us/step - loss: 472.8433 - mean_squared_error: 403.9955
Epoch 142/250
10447/10447 [==============================] - 0s 2us/step - loss: 472.4044 - mean_squared_error: 403.5016
Epoch 143/250
10447/10447 [===================

10447/10447 [==============================] - 0s 2us/step - loss: 443.1916 - mean_squared_error: 371.3515
Epoch 203/250
10447/10447 [==============================] - 0s 2us/step - loss: 442.7137 - mean_squared_error: 370.8266
Epoch 204/250
10447/10447 [==============================] - 0s 3us/step - loss: 442.2306 - mean_squared_error: 370.3014
Epoch 205/250
10447/10447 [==============================] - 0s 3us/step - loss: 441.7425 - mean_squared_error: 369.7736
Epoch 206/250
10447/10447 [==============================] - 0s 2us/step - loss: 441.2367 - mean_squared_error: 369.2225
Epoch 207/250
10447/10447 [==============================] - 0s 2us/step - loss: 440.7451 - mean_squared_error: 368.6836
Epoch 208/250
10447/10447 [==============================] - 0s 2us/step - loss: 440.2415 - mean_squared_error: 368.1334
Epoch 209/250
10447/10447 [==============================] - 0s 2us/step - loss: 439.7435 - mean_squared_error: 367.5919
Epoch 210/250
10447/10447 [===================

8631/8631 [==============================] - 0s 3us/step - loss: 3639.2451 - mean_squared_error: 3607.8016
Epoch 19/250
8631/8631 [==============================] - 0s 2us/step - loss: 3231.9252 - mean_squared_error: 3198.5249
Epoch 20/250
8631/8631 [==============================] - 0s 2us/step - loss: 2876.8062 - mean_squared_error: 2841.5576
Epoch 21/250
8631/8631 [==============================] - 0s 2us/step - loss: 2570.9785 - mean_squared_error: 2533.9933
Epoch 22/250
8631/8631 [==============================] - 0s 2us/step - loss: 2310.3989 - mean_squared_error: 2271.8184
Epoch 23/250
8631/8631 [==============================] - 0s 2us/step - loss: 2090.0989 - mean_squared_error: 2050.0575
Epoch 24/250
8631/8631 [==============================] - 0s 2us/step - loss: 1905.5180 - mean_squared_error: 1864.1675
Epoch 25/250
8631/8631 [==============================] - 0s 2us/step - loss: 1751.6713 - mean_squared_error: 1709.1493
Epoch 26/250
8631/8631 [=============================

KeyboardInterrupt: 

In [1]:
print("Total time {}".format(totalTime))
print(results)

NameError: name 'totalTime' is not defined